In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [3]:
def selectkbest(indep_X, dep_Y, n):
    # Initialize SelectKBest with the chi-squared score function and desired number of features (k=n)
    test = SelectKBest(score_func=chi2, k=n)
    
    # Fit SelectKBest using independent variables (indep_X) and dependent variable (dep_Y)
    fit1 = test.fit(indep_X, dep_Y)
    
    # Transform the independent variables to select the top k features
    selectk_features = fit1.transform(indep_X)
    
    # Return the selected top k features
    return selectk_features


In [4]:
#Standard scalar code
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    

In [5]:
#Classification codes
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm


In [6]:
def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      

In [7]:
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [8]:
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [9]:
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [10]:
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [11]:
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm 

In [12]:
def random(X_train, y_train, X_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    
    # Correct attribute for feature importances
    importances = classifier.feature_importances_
    feature_importances = pd.Series(importances, index=X_train.columns)
    # Assuming cm_prediction is defined to return these variables
    classifier, Accuracy, report, X_test, y_test, cm = cm_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm

In [13]:
def selectk_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    # Create the DataFrame
    dataframe = pd.DataFrame(index=['ChiSquare'], columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Navie', 'Decision', 'Random'])
    
    # Ensure we only iterate up to the length of the shortest list
    min_length = min(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    
    # Populate the DataFrame
    for number in range(min_length):
        dataframe.loc['ChiSquare', 'Logistic'] = acclog[number]
        dataframe.loc['ChiSquare', 'SVMl'] = accsvml[number]
        dataframe.loc['ChiSquare', 'SVMnl'] = accsvmnl[number]
        dataframe.loc['ChiSquare', 'KNN'] = accknn[number]
        dataframe.loc['ChiSquare', 'Navie'] = accnav[number]
        dataframe.loc['ChiSquare', 'Decision'] = accdes[number]
        dataframe.loc['ChiSquare', 'Random'] = accrf[number]
    
    return dataframe

In [14]:
dataset1=pd.read_csv("prep.csv",index_col=None)

df2=dataset1

In [15]:
df2 = pd.get_dummies(df2, drop_first=True)

In [16]:
indep_x=df2.drop('classification_yes',axis=1)
dep_Y=df2['classification_yes']

In [17]:
kbest=selectkbest(indep_x,dep_Y,6) 

In [18]:

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]


In [19]:
kbest

array([[3.00000000e+00, 1.48112676e+02, 5.74821053e+01, 3.07735602e+00,
        3.88689024e+01, 8.40819113e+03],
       [2.00000000e+00, 1.48112676e+02, 2.20000000e+01, 7.00000000e-01,
        3.40000000e+01, 1.23000000e+04],
       [1.00000000e+00, 9.90000000e+01, 2.30000000e+01, 6.00000000e-01,
        3.40000000e+01, 8.40819113e+03],
       ...,
       [3.00000000e+00, 1.10000000e+02, 1.15000000e+02, 6.00000000e+00,
        2.60000000e+01, 9.20000000e+03],
       [0.00000000e+00, 2.07000000e+02, 8.00000000e+01, 6.80000000e+00,
        3.88689024e+01, 8.40819113e+03],
       [0.00000000e+00, 1.00000000e+02, 4.90000000e+01, 1.00000000e+00,
        5.30000000e+01, 8.50000000e+03]])

In [20]:
X_train, X_test, y_train, y_test=split_scalar(kbest,dep_Y) 

In [21]:
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

In [22]:
classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
result=selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

In [ ]:
result
